In [23]:
import pandas as pd

In [24]:
# df = pd.read_csv('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/people_cleaner/tech4age_exports/researchers_19-11-2025.csv')
df = pd.read_csv('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/people_cleaner/tech4age_exports/researchers_08-01-2026.csv')

In [25]:
df.head(3)


,Soggetto,Nome,Cognome,Codice fiscale,Email,social media,Biosketch,Scientific-disciplinary group,Research interests,task,spoke,WP,Settore scientifico-disciplinare
0,Consiglio Nazionale delle Ricerche - CNR,Daniela,Trisciuoglio,TRSDNL73R55H501N,daniela.trisciuoglio@cnr.it,NaN,NaN,NaN,Autophagy,2; 5,2,2,BIOS-10/A
1,Consiglio Nazionale delle Ricerche - CNR,Tiziana,Tesauro,TSRTZN71H58F205V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Consiglio Nazionale delle Ricerche - CNR,Pietro,Siciliano,SCLPRL60D26I549H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.dropna(subset=['spoke'], inplace=True)

In [27]:
# registered online
sum(df['spoke'].value_counts().values)

466

In [28]:
cens_online = df[['Nome', 'Cognome', 'spoke' ,'Codice fiscale', 'Email']]

In [29]:
manual = pd.read_excel('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/people_cleaner/manually_gathered_cane.xlsx')
# manual = pd.read_excel('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/people_cleaner/manually_gathered_cleaned_19-11-2025.xlsx')

In [30]:
manual.columns

Index(['CodiceFiscale', 'Affiliation/Università', 'Nome', 'Cognome',
       'Settore scientifico-disciplinare', 'Email', 'Research Interests',
       'Spoke', 'WP', 'Task', 'Unnamed: 10'],
      dtype='object')

In [31]:
manual = manual[['Nome', 'Cognome', 'CodiceFiscale', 'Email', 'Spoke']]
manual.columns = ['Nome', 'Cognome', 'Codice fiscale', 'Email', 'spoke']

In [32]:
df = pd.concat([cens_online, manual])

In [33]:
df['full_name'] = df['Nome'] + ' ' + df['Cognome']

In [34]:
df.spoke.value_counts()

spoke
2                                                    71
1                                                    50
4                                                    47
9                                                    45
2                                                    42
                                                     ..
2; 3                                                  1
Spoke 5 - Providing care for an ageing population     1
SPOKE9                                                1
1,                                                    1
2; 8                                                  1
Name: count, Length: 68, dtype: int64

In [35]:
df.to_excel('../data/spoke_to_clean.xlsx', index=False)

In [36]:
clean = pd.read_excel('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/Mario_report/data/processed/spoke_clean.xlsx')

In [37]:
clean = clean[['spoke', 'full_name']]

In [38]:
clean

,spoke,full_name
0,2,Daniela Trisciuoglio
1,2,Serena Sanna
2,2,Francesca Cavalcanti
3,9,Paolo Barsocchi
4,2,Antonino Colanzi
...,...,...
686,3,Francesco Paolo Fanizzi
687,3,Biagio Iacolare
688,3,Marta Massei
689,2,Venturina Stagnu


In [39]:
cen = pd.read_csv('/Users/navid/Documents/1_Projects/0_Age-It/Our Tasks/Mario_report/data/processed/chord_authors.csv')

In [40]:

cen

,authors_full
0,"Mattia Stival, Lorenzo Schiavon, Gaia Bertarel..."
1,"Nicolas Chopin, Francesca R Crucinio, Sumeetpa..."
2,"Lorenzo Schiavon, Mattia Stival"
3,Lorenzo Schiavon
4,NaN
...,...
1121,"Michela Cinquini, Nicola Rocco, Giuseppe Catan..."
1122,"Peer Christiansen, Marco Mele, Anne Bodilsen, ..."
1123,"Yazan A Masannat, Nicola Rocco, Emanuele Garre..."
1124,"G. Catanuto, N. Rocco, A. Maglia, P. Barry, A...."


In [41]:
import pandas as pd

# 1) add paper id
cen = cen.reset_index().rename(columns={"index": "paper_id"})

# 2) explode authors
cen_long = (
    cen.assign(author=cen["authors_full"].str.split(","))
       .explode("author")
)
cen_long["author"] = cen_long["author"].astype(str).str.strip()

# 3) map to spoke
cen_long = cen_long.merge(
    clean[["full_name", "spoke"]],
    left_on="author",
    right_on="full_name",
    how="left"
)

# 4) count each paper once per spoke
papers_per_spoke = (
    cen_long[["paper_id", "spoke"]]
    .dropna(subset=["spoke"])
    .drop_duplicates()
    .groupby("spoke")
    .size()
    .reset_index(name="n_papers")
    .sort_values("n_papers", ascending=False)
)

papers_per_spoke['spoke'] = papers_per_spoke['spoke'].astype(int).astype(str)

In [50]:
import pandas as pd

spoke_labels = {
    '1': "DEMOGRAFIA POSITIVA",
    '2': "BIOLOGIA DELL’ INVECCHIAMENTO",
    '3': "DALLA PREVENZIONE ALLA CURA",
    '4': "INVECCHIAMENTO ATTIVO E IN BUONA SALUTE",
    '5': "SISTEMI DI CURA ALL’ANZIANO",
    '6': "SILVER ECONOMY",
    '7': "RAPPORTI TRA LE GENERAZIONI",
    '8': "RICERCA CLINICA E TECNOLOGIE AVANZATE",
    '9': "CONVIVERE CON LE TECNOLOGIE",
    '10': "STRATEGIE INTEGRATE PER L’INVECCHIAMENTO",
}

papers_per_spoke["spoke_label"] = papers_per_spoke["spoke"].map(spoke_labels)

papers_per_spoke.to_csv("papers_per_spoke.csv", index=False)

In [51]:
papers_per_spoke.to_csv('papers_per_spoke.csv', index=False)

In [52]:
papers_per_spoke

,spoke,n_papers,spoke_label
1,2,153,BIOLOGIA DELL’ INVECCHIAMENTO
2,3,112,DALLA PREVENZIONE ALLA CURA
7,8,106,RICERCA CLINICA E TECNOLOGIE AVANZATE
8,9,87,CONVIVERE CON LE TECNOLOGIE
3,4,82,INVECCHIAMENTO ATTIVO E IN BUONA SALUTE
0,1,61,DEMOGRAFIA POSITIVA
9,10,42,STRATEGIE INTEGRATE PER L’INVECCHIAMENTO
4,5,37,SISTEMI DI CURA ALL’ANZIANO
5,6,26,SILVER ECONOMY
6,7,7,RAPPORTI TRA LE GENERAZIONI
